In [1]:
import pandas as pd

# 1. 데이터 파일 로드
try:
    df = pd.read_csv('/home/user/workdir/CMAQ_Emulator/main/resources/geom/si_gun_gu_allocation_v1.csv')
    print("✅ 원본 파일('si_gun_gu_allocation_v1.csv')을 성공적으로 읽었습니다.")
except FileNotFoundError:
    print("❌ 오류: 'si_gun_gu_allocation_v1.csv' 파일을 찾을 수 없습니다. 스크립트와 동일한 위치에 파일을 놓아주세요.")
    exit()

# 광역지자체 한글-영문 변환을 위한 맵 정의
sido_korean_to_english = {
    '서울': 'Seoul', '부산': 'Busan', '대구': 'Daegu', '인천': 'Incheon',
    '광주': 'Gwangju', '대전': 'Daejeon', '울산': 'Ulsan', '세종': 'Sejong',
    '경기': 'Gyeonggi', '강원': 'Gangwon', '충북': 'Chungbuk', '충남': 'Chung-Nam',
    '전북': 'Jeon-Buk', '전남': 'Jeon-Nam', '경북': 'Gyeong-Buk', '경남': 'Gyeong-Nam',
    '제주': 'Jeju'
}

# 2. 변환 규칙 #1 적용: OOO-gu, XXX-si -> XXX-si (서울 제외)
# '광역지자체'가 '서울'이 아니고, 'SIG_ENG_NM'에 쉼표(,)가 포함된 경우를 찾습니다.
mask_gu_si = (df['광역지자체'] != '서울') & (df['SIG_ENG_NM'].str.contains(','))

# 해당 데이터의 'SIG_ENG_NM'을 쉼표로 분리하여 마지막 부분(XXX-si)만 남깁니다.
df.loc[mask_gu_si, 'SIG_ENG_NM'] = df.loc[mask_gu_si, 'SIG_ENG_NM'].str.split(',').str[-1].str.strip()
print("✅ 규칙 #1 적용 완료: '-gu, -si' 형식의 데이터를 '-si'로 통합했습니다.")


# 3. 변환 규칙 #2 적용: 동서남북 이름 형식 표준화 (예: Nam-gu -> Nam-gu, Ulsan)
# 표준화가 필요한 동서남북 형식의 이름 목록
directional_names = ['Nam-gu', 'Buk-gu', 'Dong-gu', 'Seo-gu', 'Jung-gu']

# 'SIG_ENG_NM'이 위 목록에 포함된 경우를 찾습니다.
mask_directional = df['SIG_ENG_NM'].isin(directional_names)

# 해당 데이터의 'SIG_ENG_NM' 뒤에 영문 광역지자체 이름을 붙입니다.
df.loc[mask_directional, 'SIG_ENG_NM'] = df.loc[mask_directional, 'SIG_ENG_NM'] + ', ' + df.loc[mask_directional, '광역지자체'].map(sido_korean_to_english)
print("✅ 규칙 #2 적용 완료: '동/서/남/북/중구' 이름 형식을 표준화했습니다.")


# 4. 최종 데이터 정리 및 저장
# 코드에서 사용할 최종 컬럼들을 선택하고 이름을 표준화합니다.
df['Region_Name'] = df['광역지자체'].map(sido_korean_to_english)
final_df = df[['Y(Row)', 'X(Column)', 'Region_Name', 'SIG_ENG_NM', 'ratio']].copy()
final_df.rename(columns={
    'Y(Row)': 'Row',
    'X(Column)': 'Column',
    'ratio': 'Ratio'
}, inplace=True)

# 변환된 결과를 새로운 CSV 파일로 저장
output_filename = '/home/user/workdir/CMAQ_Emulator/main/resources/geom/si_gun_gu_allocation_v2.csv'
final_df.to_csv(output_filename, index=False, encoding='utf-8-sig')
print(f"\n🎉 모든 변환이 완료되었습니다. 결과가 '{output_filename}' 파일로 저장되었습니다.")

✅ 원본 파일('si_gun_gu_allocation_v1.csv')을 성공적으로 읽었습니다.
✅ 규칙 #1 적용 완료: '-gu, -si' 형식의 데이터를 '-si'로 통합했습니다.
✅ 규칙 #2 적용 완료: '동/서/남/북/중구' 이름 형식을 표준화했습니다.

🎉 모든 변환이 완료되었습니다. 결과가 '/home/user/workdir/CMAQ_Emulator/main/resources/geom/si_gun_gu_allocation_v2.csv' 파일로 저장되었습니다.
